# Assignment 2 - Elementary Probability and Information Theory 


### Instructions and Hints:

* This notebook loads some data into a `pandas` dataframe, then does a small amount of preprocessing. Make sure your data can load by stepping through all of the cells up until question 1. 
* Most of the questions require you to write some code. In many cases, you will write some kind of probability function like we did in class using the data. 
* Some of the questions only require you to write answers, so be sure to change the cell type to markdown or raw text
* Don't worry about normalizing the text this time (e.g., lowercase, etc.). Just focus on probabilies. 
* Most questions can be answered in a single cell, but you can make as many additional cells as you need. 
* Follow the instructions on the corresponding assignment Trello card for submitting your assignment. 

In [1]:
import pandas as pd 

data = pd.read_csv('pnp-train.txt',delimiter='\t',encoding='latin-1', # utf8 encoding didn't work for this
                  names=['type','name']) # supply the column names for the dataframe

# this next line creates a new column with the lower-cased first word
data['first_word'] = data['name'].map(lambda x: x.lower().split()[0])

In [2]:
data[:10]

,type,name,first_word
0,drug,Dilotab,dilotab
1,movie,Beastie Boys: Live in Glasgow,beastie
2,person,Michelle Ford-Eriksson,michelle
3,place,Ramsbury,ramsbury
4,place,Market Bosworth,market
5,drug,Cyanide Antidote Package,cyanide
6,person,Bill Johnson,bill
7,place,Ettalong,ettalong
8,movie,The Suicide Club,the
9,place,Pézenas,pézenas


In [3]:
data.describe()

,type,name,first_word
count,21001,21001,21001
unique,5,20992,13703
top,movie,Aberdeen,the
freq,6262,2,635


## 1. Write a probability function/distribution $P(T)$ over the types. 

Hints:

* The Counter library might be useful: `from collections import Counter`
* Write a function `def P(T='')` that returns the probability of the specific value for T
* You can access the types from the dataframe by calling `data['type']`

In [4]:
from collections import Counter
types = Counter(data.type)

In [5]:
def P(T=''):
    return types[T]/len(data.type)

## 2. What is `P(T='movie')` ?

In [6]:
P(T='movie')

0.29817627732012764

## 3. Show that your probability distribution sums to one.

In [7]:
sum([P(T=t) for t in set(data.type)])

1.0

## 4. Write a joint distribution using the type and the first word of the name

Hints:

* The function is $P2(T,W_1)$
* You will need to count up types AND the first words, for example: ('person','bill)
* Using the [itertools.product](https://docs.python.org/2/library/itertools.html#itertools.product) function was useful for me here

In [8]:
values_combination = list(zip(data.type, data.first_word))
combination_count = Counter(values_combination)

In [9]:
def P2(T='', W1=''):
    return combination_count[(T, W1)]/sum(combination_count.values())

## 5. What is P2(T='person', W1='bill')? What about P2(T='movie',W1='the')?

In [10]:
P2(T='person', W1='bill')

0.00047616780153326033

In [11]:
P2(T='movie', W1='the')

0.02747488214846912

## 6. Show that your probability distribution P(T,W1) sums to one.

In [12]:
sum([P2(T=pair[0], W1=pair[1]) for pair in set(values_combination)])

0.9999999999997958

## 7. Make a new function Q(T) from marginalizing over P(T,W1) and make sure that Q(T) sums to one.

Hints:

* Your Q function will call P(T,W1)
* Your check for the sum to one should be the same answer as Question 3, only it calls Q instead of P.

In [13]:
def Q(T=''):
    return sum([P2(T, W1=word) for word in set(data.first_word)])

In [14]:
Q('movie')

0.2981762773201263

In [15]:
sum([Q(T=t) for t in set(data.type)])

1.0000000000000278

## 8. What is the KL Divergence of your Q function and your P function for Question 1?

* Even if you know the answer, you still need to write code that computes it.

In [16]:
import math

(P('movie') * math.log(P('movie')/Q('movie')) + P('drug') * math.log(P('drug')/Q('drug')) + P('place') * math.log(P('place')/Q('place')) + P('person') * math.log(P('person')/Q('person')) + P('company') * math.log(P('company')/Q('company')))

-2.7707283213696594e-14

## 9. Convert from P(T,W1) to P(W1|T) 

Hints:

* Just write a comment cell, no code this time. 
* Note that $P(T,W1) = P(W1,T)$

We know, $$ P(T,W1) = P(T)P(W1|T)$$
So by changing, $$P(W1|T) = P(T,W1) / P(T)$$

## 10. Write a function `Pwt` (that calls the functions you already have) to compute $P(W_1|T)$.

* This will be something like the multiplication rule, but you may need to change something

In [17]:
def Pwt(W1='', T=''):
    return P2(T=T, W1=W1)/P(T=T)

## 11. What is P(W1='the'|T='movie')?

In [18]:
Pwt(W1='the',T='movie')

0.09214308527626956

## 12. Use Baye's rule to convert from P(W1|T) to P(T|W1). Write a function Ptw to reflect this. 

Hints:

* Call your other functions.
* You may need to write a function for P(W1) and you may need a new counter for `data['first_word']`

In [19]:
def Pw(W1=''):
    first_word_counter = Counter(data['first_word'])
    return first_word_counter[W1]/sum(first_word_counter.values())

In [20]:
def Ptw(T='', W1=''):
    return (Pwt(W1=W1, T=T)*P(T=T))/Pw(W1=W1)

## 13 
### What is P(T='movie'|W1='the')? 
### What about P(T='person'|W1='the')?
### What about P(T='drug'|W1='the')?
### What about P(T='place'|W1='the')
### What about P(T='company'|W1='the')

In [21]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [22]:
Ptw(T='person',W1='the')

0.0

In [23]:
Ptw(T='drug',W1='the')

0.0

In [24]:
Ptw(T='place',W1='the')

0.0015748031496062992

In [25]:
Ptw(T='company',W1='the')

0.08976377952755905

## 14 Given this, if the word 'the' is found in a name, what is the most likely type?

Its most likely type is movie

## 15. Is Ptw(T='movie'|W1='the') the same as Pwt(W1='the'|T='movie') the same? Why or why not?

In [26]:
Ptw(T='movie',W1='the')

0.9086614173228347

In [27]:
Pwt(W1='the', T='movie')

0.09214308527626956

Definetly they are not same. The probability of a type being "movie" given the word "the" is different from the probability of a word being "the" given the type is a "movie".

## 16. Do you think modeling Ptw(T|W1) would be better with a continuous function like a Gaussian? Why or why not?

- Answer in a markdown cell


I dont think modeling with Ptw(T|W1) would be better with a continuous function because we are working with discrete values and Ptw(T|W1) is used in such cases. Continuous variables are better modeled through continuous probability functions like Gaussian.  

In [41]:
from client.api.notebook import Notebook
ok = Notebook('a2.ok')
ok.auth(inline=True, force=True)

Assignment: A2 Python and Jupyter
OK, version v1.18.1


Open the following URL:

https://okpy.org/client/login/

After logging in, copy the code from the web page and paste it into the box.
Then press the "Enter" key on your keyboard.

Paste your code here: vEKcYVHoicJVnwEzdu8oDu6Tpj46Rl
Successfully logged in as MostofanajmusSak@u.boisestate.edu


In [42]:
ok.submit()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Saving notebook... Saved 'A2-probability-information-theory.ipynb'.
Submit... 100% complete
Submission successful for user: MostofanajmusSak@u.boisestate.edu
URL: https://okpy.org/bsu/nlp/sp21/a2/submissions/G87YDQ

